<h3>Data Loading</h3>
<p>The Data Load feature supports loading a CSV file into an existing table in your database.  The data must be in the proper format to insert into the columns of your table.  Specifically, data types must match and any data constraints must be met.  No pre-processing of the data file is performed.  The maximum number of rows loaded is 100,000.  The status of your dataload is stored in the Teradata SQL History.  You can view the history by running the %history magic command.  If an error occurs while attempting to load your data, the entire load operation will be rolled back. 
<p>You can cancel the dataload once it has started by pressing the interrupt button on the notebook toolbar.
    
<p><b>***NOTE: To execute the SQL or magic command in the cell, press SHIFT+ENTER or the run button in the toolbar.</b>


#### Enter '%help dataload' to find out more about the %dataload magic

In [1]:
%help dataload

#### %dataload
Load a CSV file into a database table

%dataload DATABASE=\<DatabaseName\>, TABLE=\<TableName\>, FILEPATH=\<Path to CSV file\> [, BATCHSIZE=\<Rows per Batch\>]

* The **FILEPATH** is relative to the JupyterLab workspace (where JupyterLab was launched)
* You can specify an absolute file path by starting the path with the OS specific directory notation
* **BATCHSIZE** is an optional parameter. It specifies the number of rows per batch
  * Valid values are between 1 and 16383. The default value is 5000
* The maximum number of rows that the **%dataload** magic will load is currently set to 100,000 rows
* The data must be in the proper format to insert into the columns of your table. Specifically, data types must match and any data constraints must be met. 
No pre-processing of the data file is performed.
* If an error occurs while attempting to load your data, the entire load operation will be rolled back. 
* The status of your dataload is stored in the Teradata SQL History.

#### Add a connection and connect to your Vantage system
Replace the systemName, user, host, and db values for your system.

In [ ]:
%var systemName=<Vantage-system>, user=<user name>, host=<host name or IP>, db=<database>

In [ ]:
%addconnect name=${systemName}, user=${user}, host=${host}

In [ ]:
%connect ${systemName}

#### Set your Vantage Database

In [ ]:
DATABASE ${db}

#### Create an empty table in your database

In [ ]:
CREATE MULTISET TABLE airports ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
     (
      City VARCHAR(50) CHARACTER SET LATIN CASESPECIFIC,
      Name VARCHAR(50) CHARACTER SET LATIN CASESPECIFIC,
      Code VARCHAR(3) CHARACTER SET LATIN CASESPECIFIC)
PRIMARY INDEX ( City );

<h4>Run %dataload magic</h4>
<p>Run the %dataload magic, specifying the <b>DATABASE</b>, <b>TABLE</b>, and <b>FILEPATH</b> parameter values. The <b>FILEPATH</b> indicates the path to your CSV file. The path is relative to your JupyterLab workspace. You can also provide an absolute path to your CSV file in the <b>FILEPATH</b> value.
<p> Below shows loading the local airports.csv file (located in the notebooks/sql/data directory) into the airports table within your database.

In [ ]:
%dataload DATABASE=${db}, TABLE=airports, FILEPATH=notebooks/sql/data/airports.csv

<h4>Verify data was loaded by running SELECT query</h4>

In [ ]:
SELECT * FROM airports;

<h4>Data Load Status</h4>
<p>The status of the dataload is stored in the history. Run the %history magic to display the latest status.

In [ ]:
%history

#### Clean up table

In [ ]:
DROP TABLE airports;

Copyright 2020 Teradata. All rights reserved.